# Precipitation at All CROCUS Sites

In [ ]:
from IPython.display import IFrame
import pandas as pd
import sage_data_client
import holoviews as hv
import hvplot.pandas
from metpy.units import units
import metpy.calc as mpcalc
from bokeh.models import DatetimeTickFormatter
import sage_data_client

hv.extension("bokeh")

def apply_formatter(plot, element):
    plot.handles['xaxis'].formatter = DatetimeTickFormatter(hours='%m/%d/%Y \n %l:%M %p',
                                                            minutes='%m/%d/%Y \n %l:%M %p',
                                                            hourmin='%m/%d/%Y \n %l:%M %p',
                                                            days='%m/%d/%Y \n %l:%M %p',
                                                            months='%m/%d/%Y \n %l:%M %p')

def add_local_time(df):
    df["Local_Time"] = pd.to_datetime(df.index,unit='s', utc=True)\
                          .map(lambda x: x.tz_convert('America/Chicago'))
    return df

def search_convert_to_inch(site):

    subset = sage_data_client.query(
                start="-1d",
                filter={
                    "plugin": "registry.sagecontinuum.org/jrobrien/waggle-wxt536:0.24.11.14.*",
                    "vsn": "W096",
                    "name": "wxt.rain.accumulation"
                }
            ).set_index("timestamp")

    subset["value"] = (subset.value.values * units.mm).to("in")

    if len(subset) > 0:
        subset["value"] = subset.value - subset.value.values[0]
    else:
        pass
    return subset.value

import warnings
warnings.filterwarnings('ignore')

In [ ]:
merged = pd.DataFrame({"ATMOS": search_convert_to_inch("W0A4"),
                       "HUM": search_convert_to_inch("W0A1"),
                       "BIG": search_convert_to_inch("W0A0"),
                       "SHEDD": search_convert_to_inch("W09E"),
                       "DOWN": search_convert_to_inch("W09D"),
                       "NU": search_convert_to_inch("W099"),
                       "UIC": search_convert_to_inch("W096"),
                       "VLPK": search_convert_to_inch("W095"),
                       "CSU": search_convert_to_inch("W08E"),
                       "NEIU": search_convert_to_inch("W08D"),
                       "CCICS": search_convert_to_inch("W08B"),}
)

# Deal with negavtive values
for column in merged.columns:
    if merged[column].min() < 0:
        merged = merged.drop(columns=[column])

In [ ]:
merged.resample("1min").mean().hvplot(title= "12 Hour Precipitation Accumulation at \n CROCUS Sites [inches]",
                                      ylabel="Precipitation Accumulation [inches]",
                                      xlabel="Local Time (Central/US)"
                                     ).opts(hooks=[apply_formatter])